# MNIST 手写数字识别
**自动计算准确度（metrics）**

In [5]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, optimizers, datasets, Sequential, metrics

**加载数据**

In [2]:
(xs, ys),_ = datasets.mnist.load_data()
print('datasets:', xs.shape, ys.shape, xs.min(), xs.max())

batch_size = 32

xs = tf.convert_to_tensor(xs, dtype=tf.float32) / 255.
db = tf.data.Dataset.from_tensor_slices((xs,ys))
db = db.batch(batch_size).repeat(30)

datasets: (60000, 28, 28) (60000,) 0 255


**模型**

In [6]:
model = Sequential([
    layers.Dense(256, activation='relu'), 
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])
model.build(input_shape=(4, 28*28))
model.summary()

optimizer = optimizers.SGD(lr=0.01)
acc_meter = metrics.Accuracy()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              multiple                  200960    
_________________________________________________________________
dense_4 (Dense)              multiple                  32896     
_________________________________________________________________
dense_5 (Dense)              multiple                  1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


**训练模型**

In [8]:
for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # 打平操作，[b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # Step1. 得到模型输出output [b, 784] => [b, 10]
        out = model(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # 计算差的平方和，[b, 10]
        loss = tf.square(out-y_onehot)
        # 计算每个样本的平均误差，[b]
        loss = tf.reduce_sum(loss) / x.shape[0]


    acc_meter.update_state(tf.argmax(out, axis=1), y)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))


    if step % 2000==0:

        print(step, 'loss:', float(loss), 'acc:', acc_meter.result().numpy())
        acc_meter.reset_states()

0 loss: 0.0978606790304184 acc: 0.9759489
2000 loss: 0.09753087908029556 acc: 0.96928126
4000 loss: 0.0901997983455658 acc: 0.9710938
6000 loss: 0.08087755739688873 acc: 0.97223437
8000 loss: 0.09405967593193054 acc: 0.97353125
10000 loss: 0.11057624220848083 acc: 0.9750469
12000 loss: 0.046288274228572845 acc: 0.97589064
14000 loss: 0.03741767257452011 acc: 0.9768906
16000 loss: 0.07959193736314774 acc: 0.978
18000 loss: 0.060375623404979706 acc: 0.97915626
20000 loss: 0.056397806853055954 acc: 0.9796719
22000 loss: 0.04052315279841423 acc: 0.9808281
24000 loss: 0.07147417962551117 acc: 0.98164064
26000 loss: 0.053170979022979736 acc: 0.98240626
28000 loss: 0.06844788044691086 acc: 0.9832031
30000 loss: 0.03784005716443062 acc: 0.98428124
32000 loss: 0.050008852034807205 acc: 0.9846875
34000 loss: 0.04482151195406914 acc: 0.9852656
36000 loss: 0.047730326652526855 acc: 0.98592186
38000 loss: 0.04797711968421936 acc: 0.9863125
40000 loss: 0.08105066418647766 acc: 0.98675
42000 loss: 0.